In [3]:
import os, h5py, click, math, pickle
from pandas import read_pickle
import numpy as np

coef_N=read_pickle('coeffsN.pkl')
coef_Npi=read_pickle('coeffsNpi.pkl')

max_mom2=1
momDic={}
for mom2 in range(max_mom2+1):
    momDic[mom2]=[]
t=math.floor(np.sqrt(max_mom2))
for x in range(-t,t+1):
    for y in range(-t,t+1):
        for z in range(-t,t+1):
            mom2=x**2+y**2+z**2
            if mom2 in momDic:
                momDic[mom2].append([x,y,z])
ops=[]                
for l in ['l1','l2']:
    for mom2,moms in momDic.items():
        for x,y,z in moms:
            op_base='g;{},{},{};'.format(x,y,z)
            op_base+='{};'.format('G1g' if mom2==0 else 'G1' if mom2 in [1,4] else 'G' if mom2 in [2,3] else 'NA')
            op=op_base+'a;{};N'.format(l); ops.append(op)
            occs = {0:['N1pi1,a'],1:['N1pi0,a','N0pi1,a']}[mom2]
            for occ in occs:
                op=op_base+'{};{};N,pi'.format(occ,l); ops.append(op)
ops.append('g;0,0,0;G1u;N0pi0,a;l1;N,pi'); ops.append('g;0,0,0;G1u;N0pi0,a;l2;N,pi');        

ops.sort()
coeffs={}
for op in ops:
    coeffs[op]=[]
    _,pt,irrep,occ,l,fla=op.split(';')
    pt2=np.sum([int(i)**2 for i in pt.split(',')])
    group={0:'2Oh',1:'2C4v'}[pt2]
    mom=tuple([int(i) for i in pt.split(',')])
    print(op)
    if fla=='N':
        val=coef_N[group][irrep][l][mom][occ]['cg5']
        for a in [0,1,2,3]:
            if np.abs(val[a])<1e-4:
                continue
            opt='t;{};{};N'.format(pt,a); coeffs[op].append((opt,val[a]))
        coeffs[op].sort()
        print(len(coeffs[op]),coeffs[op])
    else:
        occ1,occ2=occ.split(',')
        occ1=occ1.replace('pi','\\pi')
        for momNpi,val in coef_Npi[group][irrep][l][mom][occ1][occ2].items():
            val=val['cg5;g5']
            for a in [0,1,2,3]:
                if np.abs(val[a])<1e-4:
                    continue
                momN=','.join([str(temp) for temp in momNpi[0]])
                mompi=','.join([str(temp) for temp in momNpi[1]])
                opt='t;{},{};{};N,pi'.format(momN,mompi,a); coeffs[op].append((opt,-1j*val[a] if occ!='N0pi0,a' else val[a])) # -1j is used to make N-Npi real
        coeffs[op].sort()
        print(len(coeffs[op]),coeffs[op])
    print()
    # break

with open('group_coeffs.pkl','wb') as f:
    pickle.dump(coeffs,f)

g;-1,0,0;G1;N0pi1,a;l1;N,pi
2 [('t;0,0,0,-1,0,0;0;N,pi', -0.7071067811865476j), ('t;0,0,0,-1,0,0;1;N,pi', 0.7071067811865476j)]

g;-1,0,0;G1;N0pi1,a;l2;N,pi
2 [('t;0,0,0,-1,0,0;0;N,pi', 0.7071067811865476j), ('t;0,0,0,-1,0,0;1;N,pi', 0.7071067811865476j)]

g;-1,0,0;G1;N1pi0,a;l1;N,pi
2 [('t;-1,0,0,0,0,0;0;N,pi', -0.7071067811865476j), ('t;-1,0,0,0,0,0;1;N,pi', 0.7071067811865476j)]

g;-1,0,0;G1;N1pi0,a;l2;N,pi
2 [('t;-1,0,0,0,0,0;0;N,pi', 0.7071067811865476j), ('t;-1,0,0,0,0,0;1;N,pi', 0.7071067811865476j)]

g;-1,0,0;G1;a;l1;N
2 [('t;-1,0,0;0;N', (0.7071067811865476+0j)), ('t;-1,0,0;1;N', (-0.7071067811865476+0j))]

g;-1,0,0;G1;a;l2;N
2 [('t;-1,0,0;0;N', (0.7071067811865476+0j)), ('t;-1,0,0;1;N', (0.7071067811865476+0j))]

g;0,-1,0;G1;N0pi1,a;l1;N,pi
2 [('t;0,0,0,0,-1,0;0;N,pi', -0.7071067811865476j), ('t;0,0,0,0,-1,0;1;N,pi', (0.7071067811865476-0j))]

g;0,-1,0;G1;N0pi1,a;l2;N,pi
2 [('t;0,0,0,0,-1,0;0;N,pi', (-0.7071067811865476+0j)), ('t;0,0,0,0,-1,0;1;N,pi', 0.7071067811865476j)]

g

In [8]:
import os, h5py, click, math, pickle
from pandas import read_pickle
import numpy as np

c=read_pickle('/project/s1174/lyan/code/projectData/NST_a/cA211.530.24/data_aux/group_coeffs.pkl')

for t in c.keys():
    if 'G1u' in t:
        print(t)

g;0,0,0;G1u;N0pi0,a;l1;N,pi
g;0,0,0;G1u;N0pi0,a;l2;N,pi
